In [1]:
import sys
sys.path.append('../../')

In [2]:
# importnb allows us to load the code from other notebooks
from importnb import Notebook
with Notebook():
    import OfficeAutomation.common as common  # will probably become OfficeAutomation.common
    import OfficeAutomation.structure as structure

In [3]:
import crestdsl.model as crest
import crestdsl.simulation as crestsim
import crestdsl.ui as ui

In [4]:
class OfficeLightControllerTest(common.TestClass):

    def test_correct_output(self):
        threshold = 500
        lc = structure.OfficeLightController()
        lc.light_threshold.value = threshold
        
        sim = crestsim.Simulator(lc)
        
        "(presence, switch, luminosity, expected_output)"
        params = [
            # light above threshold
            ("no presence", "on", threshold + 100, "on"),
            ("no presence", "off", threshold + 100, "off"),
            ("no presence", "auto", threshold + 100, "off"),
            ("detected", "on", threshold + 100, "on"),
            ("detected", "off", threshold + 100, "off"),
            ("detected", "auto", threshold + 100, "off"),
            
            # light below threshold
            ("no presence", "on", threshold - 100, "on"),
            ("no presence", "off", threshold - 100, "off"),
            ("no presence", "auto", threshold - 100, "off"),
            ("detected", "on", threshold - 100, "on"),
            ("detected", "off", threshold - 100, "off"),
            ("detected", "auto", threshold - 100, "on")
        ]
        
        for presence, switch, luminosity, expected_output in params:
            with self.subTest(presence=presence, switch_state=switch, luminosity=luminosity, expected_output=expected_output):
                lc.presence.value = presence
                lc.switch_in.value = switch
                lc.luminosity_in.value = luminosity
                sim.stabilise()
                self.assertEqual(lc.out.value, expected_output, "The output didn't match what we expected")

# OfficeLightControllerTest.runall()


In [5]:
class RoomTest(common.TestClass):
        
    def test_correct_req_electricity_calculations(self):
        room = structure.Room(number_lamps=2)
        
        ("presence", "lightswitch", "ac_setting", "expected_req_electricity")
        params = [
            ("no presence", "off", "off", 0),
            ("detected", "off", "off", 0),
            ("no presence", "on", "off", 200),
            ("detected", "on", "off", 200),
            ("no presence", "auto", "off", 0),
            ("detected", "auto", "off", 200),
            
            ("no presence", "off", "one", 500),
            ("detected", "off", "one", 500),
            ("no presence", "on", "one", 700),
            ("detected", "on", "one", 700),
            ("no presence", "auto", "one", 500),
            ("detected", "auto", "one", 700),            
        ]
        
        for pres, sw, ac_setting, expected_req in params:
            with self.subTest(presence=pres, light_switch=sw, ac_setting=ac_setting, expected_req=expected_req):
                room.presence.value = pres
                room.lightswitch.value = sw
                room.ac_setting_in.value = ac_setting
                
                sim = crestsim.Simulator(room)
                sim.stabilise()
        
                self.assertEqual(expected_req, room.req_electricity_out.value)
            
    def test_correct_states(self):
        room = structure.Room(number_lamps=2)
        room.electricity_in.value = 1000
        
        ("presence", "lightswitch", "ac_setting", "expected_ac_output", "expected_ac_state_name", "expected_lamp_state_name")
        params = [
            # ac off
            ("no presence", "off", "off", 0, "off", "off"),
            ("detected",    "off", "off", 0, "off", "off"),
            ("no presence", "on",  "off", 0, "off", "on"),
            ("detected",    "on",  "off", 0, "off", "on"),
            ("no presence", "auto","off", 0, "off", "off"),
            ("detected",    "auto","off", 0, "off", "on"),
            
            # ac on
            ("no presence", "off", "one", 500*0.35, "one", "off"),
            ("detected",    "off", "one", 500*0.35, "one", "off"),
            ("no presence", "on",  "one", 500*0.35, "one", "on"),
            ("detected",    "on",  "one", 500*0.35, "one", "on"),
            ("no presence", "auto","one", 500*0.35, "one", "off"),
            ("detected",    "auto","one", 500*0.35, "one", "on"),            
        ]
        
        for pres, sw, ac_setting, exp_ac_out, exp_ac_state, exp_lamp_state in params:
            with self.subTest(presence=pres, light_switch=sw, ac_setting=ac_setting, expected_ac_output=exp_ac_out, expected_ac_state=exp_ac_state, expected_lamp_state=exp_lamp_state):
                room.presence.value = pres
                room.lightswitch.value = sw
                room.ac_setting_in.value = ac_setting
                
                sim = crestsim.Simulator(room)
                sim.stabilise()
        
                self.assertEqual(exp_ac_out, room.acunit.output.value)
                self.assertEqual(exp_ac_state, room.acunit.current._name)
                self.assertEqual(exp_lamp_state, room.lamp_1.current._name)
                self.assertEqual(exp_lamp_state, room.lamp_2.current._name)

    def test_temperature_changes(self):
        room = structure.Room(volume=100, number_windows=2)
        room.lightswitch.value = "off"  # turn off lights
        room.electricity_in.value = 10000  # assume there's enough electricity available
        
        # temperature outside, temperature inside, ac_setting, time advance, expected temperature after 10 minutes, expected temperature after 60 minutes
        params = [
            (25, 25, "off", 10, 25),
            (25, 25, "off", 60, 25),
            (25, 25, "off", 240, 25),
            
            (30, 25, "off", 10, 25.550640151490672),
            (30, 25, "off", 60, 29.889440907723404),
            (30, 25, "off", 240, 30.000000013825833),
            
            (25, 25, "one", 10, 23.868990299168225),
            (25, 25, "one", 60, 14.957134285779393),
            (25, 25, "one", 240, 14.730046919958639),
            
            (30, 25, "one", 10, 24.419630450658893),
            (30, 25, "one", 60, 19.846575193502797),
            (30, 25, "one", 240, 19.730046933784507),
            
            (30, 25, "two", 10, 23.967226570326183),
            (30, 25, "two", 60, 15.829428907814556),
            (30, 25, "two", 240, 15.622065701767967),
            
            (30, 25, "three", 10, 23.514822689993473),
            (30, 25, "three", 60, 11.812282622126311),
            (30, 25, "three", 240, 11.514084469751426),
        ]
        
        for outside, inside, acsetting, time_advance, expected_temp in params:
            with self.subTest(outside_temperature=outside, inside_temperature_at_start=inside, ac_setting=acsetting, 
                              time_advance=time_advance, expected_temperature=expected_temp):
                room.temperature_outside.value = outside
                room.temperature.value = inside  # initial value
                room.ac_setting_in.value = acsetting

                sim = crestsim.Simulator(room, max_step_size=10)
                sim.stabilise()
                
                sim.advance(time_advance)
                self.assertAlmostEqual(room.temperature.value, expected_temp, msg=f"Temperature after {time_advance} minutes matches")
                
RoomTest.runall()

...
----------------------------------------------------------------------
Ran 3 tests in 43.250s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [6]:
class FloorTest(common.TestClass):
        
    def test_presence_produces_correct_lighting_setting(self):
        floor = structure.Floor()        
        
        floor.lightswitch_office1.value = "auto"  # set all offices to auto
        floor.lightswitch_office2.value = "auto"
        floor.lightswitch_office3.value = "auto"
        
        floor.presence_hallway.value = "no presence" # set no presence in any office
        floor.presence_office1.value = "no presence"
        floor.presence_office2.value = "no presence"
        floor.presence_office3.value = "no presence"
        
        ("hallway", "office1", "office2", "office3", "hallway_switch", "office1_switch", "office2_switch", "office3_switch")
        params = [
            ("no presence", "no presence", "no presence", "no presence", "off", "off", "off", "off"),
            ("detected",    "no presence", "no presence", "no presence", "on", "off", "off", "off"),
            ("no presence", "detected",    "no presence", "no presence", "on", "on",  "off", "off"),
            ("no presence", "no presence", "detected",    "no presence", "on", "off", "on",  "off"),
            ("no presence", "no presence", "no presence", "detected",    "on", "off", "off", "on"),
        ]
        
        for hw, o1, o2, o3, exp_hall, exp_off1, exp_off2, exp_off3 in params:
            with self.subTest(hw_presence=hw, o1_presence=o1, o2_presence=o2, o3_presence=o3, expected_hallway=exp_hall, expected_office1=exp_off1, expected_office2=exp_off2, expected_office3=exp_off3):
                floor.presence_hallway.value = hw
                floor.presence_office1.value = o1
                floor.presence_office2.value = o2
                floor.presence_office3.value = o3

                sim = crestsim.Simulator(floor)
                sim.stabilise()

                floor.hallway_light_controls.value = exp_hall
                floor.office1.light_controls.value = exp_off1
                floor.office2.light_controls.value = exp_off2
                floor.office3.light_controls.value = exp_off3

  
    
    def test_too_much_power_drawn(self):
        floor = structure.Floor()
        floor.presence_office1.value = "detected"
        floor.presence_office2.value = "detected"
        floor.presence_office3.value = "detected"
        
        floor.ac_setting_hallway.value = "one"
        floor.ac_setting_office1.value = "one"
        floor.ac_setting_office2.value = "one"
        floor.ac_setting_office3.value = "one"
        
        floor.lightswitch_office1.value = "on"
        floor.lightswitch_office2.value = "on"
        floor.lightswitch_office3.value = "on"
        
        sim = crestsim.Simulator(floor)
        sim.stabilise()
        
        
        self.assertEqual(floor.mains.current, floor.mains.no_power)
        self.assertEqual(floor.hallway.electricity_in.value, 0)
        self.assertEqual(floor.office1.electricity_in.value, 0)
        self.assertEqual(floor.office2.electricity_in.value, 0)
        self.assertEqual(floor.office3.electricity_in.value, 0)
        
    def test_surcharge_turns_on_emergency_lights(self):
        pass
        
# FloorTest.runall()